In this tutorial we will learn to:
- Instantiate a DeepPrintExtractor
- Prepare a training dataset
- Train a DeepPrintExtractor

## Instantiate a DeepPrintExtractor

This package implements a number of variants of the DeepPrint architecture. The wrapper class for all these variants is called `DeepPrintExtractor`.
It has a `fit` method to train (and save) the model as well as an `extract` method to extract the DeepPrint features for fingerprint images. 

You can also try to implement your own models, but currently this is not directly supported by the package.

In [2]:
# GPU Detection and Setup
import torch

print("=" * 80)
print("DEVICE DETECTION")
print("=" * 80)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("\n⚠️  WARNING: CUDA not available! Training will run on CPU (VERY SLOW!)")
    print("\nPossible reasons:")
    print("1. PyTorch was installed without CUDA support")
    print("2. NVIDIA drivers are not installed or outdated")
    print("3. CUDA toolkit version mismatch")
    print("\nTo fix:")
    print("  pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121")
print("=" * 80)

DEVICE DETECTION
PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
Number of GPUs: 1
Current GPU: NVIDIA GeForce RTX 2080
GPU Memory: 8.00 GB


## Training the model

Instantiating the model was easy. To train it, first we will load the training data (see the [data tutorial](./dataset_tutorial.ipynb) for how to implement your own dataset).

Besides the fingerprint images, we also need a mapping from subjects to integer labels (for pytorch). For some variants we also need minutiae data. To see how a more complex dataset can be loaded, have a look at `flx/setup/datasets.py`.

Finally, we call the `fit` method, which trains the model and saves it to the specified path.

There is also the option to add a validation set, which will be used to evaluate the embeddings during training. This is useful to monitor the training progress and to avoid overfitting.
In this example we will not use a validation set for simplicity.

In [3]:
import os
import torch 

from flx.data.dataset import *
from flx.data.image_loader import SOCOFingLoader, CrossmatchLoader
from flx.data.transformed_image_loader import TransformedImageLoader
from flx.data.label_index import LabelIndex
from flx.image_processing.binarization import LazilyAllocatedBinarizer
from flx.image_processing.augmentation import RandomPoseTransform
from flx.data.image_helpers import pad_and_resize_to_deepprint_input_size
from flx.extractor.fixed_length_extractor import get_DeepPrint_Tex

# Define paths
SOCO_DIR = os.path.abspath("dataset/soco")
CROSSMATCH_DIR = os.path.abspath("dataset/crossmatch")
STAGE1_MODEL_DIR = os.path.abspath("trained_models/stage1_soco")
STAGE2_MODEL_DIR = os.path.abspath("trained_models/stage2_crossmatch")

# Create output directories
os.makedirs(STAGE1_MODEL_DIR, exist_ok=True)
os.makedirs(STAGE2_MODEL_DIR, exist_ok=True)

print("=" * 80)
print("TWO-STAGE TRAINING STRATEGY")
print("=" * 80)
print("\nStage 1: Feature Learning on SOCOFing (6000 subjects, 1 impression each)")
print("Stage 2: Fine-tuning on Crossmatch (77 subjects, 8 impressions each)")
print("\n" + "=" * 80)

TWO-STAGE TRAINING STRATEGY

Stage 1: Feature Learning on SOCOFing (6000 subjects, 1 impression each)
Stage 2: Fine-tuning on Crossmatch (77 subjects, 8 impressions each)



## Stage 1: Feature Learning on SOCOFing

In this stage, we train on the SOCOFing dataset (6000 subjects, 1 impression each).

**Goal**: Learn discriminative features that distinguish between different fingers.

**Why this works**: Even with only 1 impression per subject, the classification loss teaches the model to extract features that separate different fingerprints. The large number of subjects (6000) provides excellent diversity.

In [4]:
# STAGE 1: Load SOCOFing dataset
print("\n--- Loading SOCOFing Dataset ---")
soco_loader = SOCOFingLoader(SOCO_DIR)
print(f"Total samples: {len(soco_loader.ids)}")
print(f"Total subjects: {soco_loader.ids.num_subjects}")
print(f"Impressions per subject: 1")

# Apply preprocessing and augmentation for training
soco_image_loader = TransformedImageLoader(
    images=soco_loader,
    poses=RandomPoseTransform(),  # Random rotation and translation
    transforms=[
        LazilyAllocatedBinarizer(ridge_width=5.0),  # Gabor filtering + binarization
        pad_and_resize_to_deepprint_input_size
    ]
)

# Create datasets
soco_fingerprints = Dataset(soco_image_loader, soco_loader.ids)
soco_labels = Dataset(LabelIndex(soco_loader.ids), soco_loader.ids)

print(f"\nStage 1 Training Set: {len(soco_fingerprints.ids)} samples")
print(f"Number of classes (subjects): {soco_fingerprints.ids.num_subjects}")


--- Loading SOCOFing Dataset ---
Created IdentifierSet with 6000 subjects and a total of 6000 samples.
Total samples: 6000
Total subjects: 6000
Impressions per subject: 1

Stage 1 Training Set: 6000 samples
Number of classes (subjects): 6000


In [5]:
# STAGE 1: Create and train the model
print("\n--- Stage 1: Training on SOCOFing ---")

stage1_extractor = get_DeepPrint_Tex(
    num_training_subjects=soco_loader.ids.num_subjects,
    num_texture_dims=512
)
print("=" * 80)

stage1_extractor.fit(
    fingerprints=soco_fingerprints,
    minutia_maps=None,
    labels=soco_labels,
    validation_fingerprints=None,  # No validation in Stage 1
    validation_benchmark=None,
    num_epochs=40,
    out_dir=STAGE1_MODEL_DIR
)

print("\n" + "=" * 80)
print("Stage 1 Complete! Model saved to:", STAGE1_MODEL_DIR)
print("=" * 80)


--- Stage 1: Training on SOCOFing ---
Using device cuda:0
No model file found at c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco\model.pyt


 --- Starting Epoch 1 of 40 ---

Training:
Using device cuda:0
No model file found at c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco\model.pyt


 --- Starting Epoch 1 of 40 ---

Training:


100%|██████████| 750/750 [11:31<00:00,  1.08it/s]



Average Loss: 13.307163186391195
Multiclass accuracy: 0.0
TrainingLogEntry(
    epoch=1,
    training_loss=13.307163186391195,
    loss_statistics={'crossent_loss_sum': 1.4132321206728617, 'center_loss_sum': 0.25016327770551045},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}
TrainingLogEntry(
    epoch=1,
    training_loss=13.307163186391195,
    loss_statistics={'crossent_loss_sum': 1.4132321206728617, 'center_loss_sum': 0.25016327770551045},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}


 --- Starting Epoch 2 of 40 ---

Training:


 --- Starting Epoch 2 of 40 ---

Training:


100%|██████████| 750/750 [11:19<00:00,  1.10it/s]



Average Loss: 13.301856862386067
Multiclass accuracy: 0.0
TrainingLogEntry(
    epoch=2,
    training_loss=13.301856862386067,
    loss_statistics={'crossent_loss_sum': 0.7076156949202219, 'center_loss_sum': 0.12375035820404688},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}
TrainingLogEntry(
    epoch=2,
    training_loss=13.301856862386067,
    loss_statistics={'crossent_loss_sum': 0.7076156949202219, 'center_loss_sum': 0.12375035820404688},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}


 --- Starting Epoch 3 of 40 ---

Training:


 --- Starting Epoch 3 of 40 ---

Training:


...

Average Loss: 9.495313529968263
Multiclass accuracy: 0.0
TrainingLogEntry(
    epoch=38,
    training_loss=9.495313529968263,
    loss_statistics={'crossent_loss_sum': 0.027970371911400244, 'center_loss_sum': 0.003264212076601229},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}
TrainingLogEntry(
    epoch=38,
    training_loss=9.495313529968263,
    loss_statistics={'crossent_loss_sum': 0.027970371911400244, 'center_loss_sum': 0.003264212076601229},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}


 --- Starting Epoch 39 of 40 ---

Training:


 --- Starting Epoch 39 of 40 ---

Training:


100%|██████████| 750/750 [11:09<00:00,  1.12it/s]



Average Loss: 9.431253532409668
Multiclass accuracy: 0.0
TrainingLogEntry(
    epoch=39,
    training_loss=9.431253532409668,
    loss_statistics={'crossent_loss_sum': 0.027107792183884188, 'center_loss_sum': 0.0031205845131323886},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}
TrainingLogEntry(
    epoch=39,
    training_loss=9.431253532409668,
    loss_statistics={'crossent_loss_sum': 0.027107792183884188, 'center_loss_sum': 0.0031205845131323886},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}


 --- Starting Epoch 40 of 40 ---

Training:


 --- Starting Epoch 40 of 40 ---

Training:


100%|██████████| 750/750 [11:09<00:00,  1.12it/s]



Average Loss: 9.396391512552897
Multiclass accuracy: 0.0
TrainingLogEntry(
    epoch=40,
    training_loss=9.396391512552897,
    loss_statistics={'crossent_loss_sum': 0.02637725165486336, 'center_loss_sum': 0.002986471762508154},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}
TrainingLogEntry(
    epoch=40,
    training_loss=9.396391512552897,
    loss_statistics={'crossent_loss_sum': 0.02637725165486336, 'center_loss_sum': 0.002986471762508154},
    training_accuracy=0.0,
    validation_equal_error_rate=0.0,
}

Stage 1 Complete! Model saved to: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco

Stage 1 Complete! Model saved to: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco


## Stage 2: Fine-tuning on Crossmatch

In this stage, we load the trained model from Stage 1 and fine-tune it on the Crossmatch dataset (77 subjects, 8 impressions each).

**Goal**: Learn intra-class similarity - that multiple impressions of the same finger should have similar embeddings.

**Why this works**: The center loss now has multiple impressions per subject to work with, so it can pull embeddings of the same finger closer together. The model already knows how to extract discriminative features from Stage 1.

In [6]:
# STAGE 2: Load Crossmatch dataset
print("\n--- Loading Crossmatch Dataset ---")
crossmatch_loader = CrossmatchLoader(CROSSMATCH_DIR)
print(f"Total samples: {len(crossmatch_loader.ids)}")
print(f"Total subjects: {crossmatch_loader.ids.num_subjects}")

# Analyze impression distribution
from collections import defaultdict
impression_counts = defaultdict(int)
for identifier in crossmatch_loader.ids:
    impression_counts[identifier.subject] += 1

avg_impressions = sum(impression_counts.values()) / len(impression_counts)
print(f"Average impressions per subject: {avg_impressions:.1f}")

# Apply preprocessing and augmentation
crossmatch_image_loader = TransformedImageLoader(
    images=crossmatch_loader,
    poses=RandomPoseTransform(),  # Same augmentation as Stage 1
    transforms=[
        LazilyAllocatedBinarizer(ridge_width=5.0),
        pad_and_resize_to_deepprint_input_size
    ]
)

# Create datasets
crossmatch_fingerprints = Dataset(crossmatch_image_loader, crossmatch_loader.ids)
crossmatch_labels = Dataset(LabelIndex(crossmatch_loader.ids), crossmatch_loader.ids)

print(f"\nStage 2 Training Set: {len(crossmatch_fingerprints.ids)} samples")
print(f"Number of classes (subjects): {crossmatch_fingerprints.ids.num_subjects}")


--- Loading Crossmatch Dataset ---
Created IdentifierSet with 77 subjects and a total of 616 samples.
Total samples: 616
Total subjects: 77
Average impressions per subject: 8.0

Stage 2 Training Set: 616 samples
Number of classes (subjects): 77


In [8]:
# STAGE 2: Load Stage 1 model and fine-tune
print("\n--- Stage 2: Fine-tuning on Crossmatch ---")

# Create new extractor with Crossmatch dimensions
stage2_extractor = get_DeepPrint_Tex(
    num_training_subjects=crossmatch_loader.ids.num_subjects,
    num_texture_dims=512
)

# Load the best model from Stage 1 (only feature extractor, not classification head)
print(f"Loading best model from Stage 1: {STAGE1_MODEL_DIR}")
from flx.setup.paths import get_best_model_file
import torch
from flx.models.torch_helpers import get_device

model_path = get_best_model_file(STAGE1_MODEL_DIR)
checkpoint = torch.load(model_path, map_location=get_device())
stage1_state = checkpoint["model_state_dict"]

# Load only the feature extractor layers (exclude classification head with mismatched size)
stage2_state = stage2_extractor.model.state_dict()
pretrained_dict = {k: v for k, v in stage1_state.items() 
                   if k in stage2_state and v.shape == stage2_state[k].shape}

print(f"Loaded {len(pretrained_dict)}/{len(stage2_state)} layers from Stage 1")
print(f"Skipped classification head (different classes: 6000 -> 77)")
stage2_extractor.model.load_state_dict(pretrained_dict, strict=False)

print(f"\nModel: DeepPrint_Tex_512 (pre-trained from Stage 1)")
print(f"Learning rate: 0.025 (default - consider reducing if unstable)")
print(f"Number of epochs: 30")
print(f"Expected time: ~5-10 min/epoch on GPU (smaller dataset)")
print("\nStarting fine-tuning...")
print("Monitor: Center loss should decrease as model learns intra-class similarity")
print("=" * 80)

stage2_extractor.fit(
    fingerprints=crossmatch_fingerprints,
    minutia_maps=None,
    labels=crossmatch_labels,
    validation_fingerprints=None,
    validation_benchmark=None,
    num_epochs=30,
    out_dir=STAGE2_MODEL_DIR
)

print("\n" + "=" * 80)
print("Stage 2 Complete! Final model saved to:", STAGE2_MODEL_DIR)
print("=" * 80)
print("\nTraining Summary:")
print(f"  Stage 1: Trained on {len(soco_fingerprints.ids)} SOCOFing samples")
print(f"  Stage 2: Fine-tuned on {len(crossmatch_fingerprints.ids)} Crossmatch samples")
print(f"  Final model location: {STAGE2_MODEL_DIR}/best_model.pyt")
print("\nNext steps:")
print("  1. Evaluate the model on held-out test data")
print("  2. Extract embeddings for your fingerprint database")
print("  3. Use for fingerprint matching/verification")
print("=" * 80)


--- Stage 2: Fine-tuning on Crossmatch ---
Loading best model from Stage 1: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco
Loading best model from Stage 1: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage1_soco


C:\Users\koechian\AppData\Local\Temp\ipykernel_23332\3561686268.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=get_dev

Loaded 896/898 layers from Stage 1
Skipped classification head (different classes: 6000 -> 77)

Model: DeepPrint_Tex_512 (pre-trained from Stage 1)
Learning rate: 0.025 (default - consider reducing if unstable)
Number of epochs: 30
Expected time: ~5-10 min/epoch on GPU (smaller dataset)

Starting fine-tuning...
Monitor: Center loss should decrease as model learns intra-class similarity
Using device cuda:0
No model file found at c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage2_crossmatch\model.pyt


 --- Starting Epoch 1 of 30 ---

Training:


100%|██████████| 77/77 [01:17<00:00,  1.01s/it]



Average Loss: 6.1834256803834595
Multiclass accuracy: 0.027597401291131973
TrainingLogEntry(
    epoch=1,
    training_loss=6.1834256803834595,
    loss_statistics={'crossent_loss_sum': 0.5406462008302862, 'center_loss_sum': 0.23228201192694825},
    training_accuracy=0.027597401291131973,
    validation_equal_error_rate=0.027597401291131973,
}
TrainingLogEntry(
    epoch=1,
    training_loss=6.1834256803834595,
    loss_statistics={'crossent_loss_sum': 0.5406462008302862, 'center_loss_sum': 0.23228201192694825},
    training_accuracy=0.027597401291131973,
    validation_equal_error_rate=0.027597401291131973,
}


 --- Starting Epoch 2 of 30 ---

Training:


 --- Starting Epoch 2 of 30 ---

Training:


...

Average Loss: 3.608847066953585
Multiclass accuracy: 0.034090906381607056
TrainingLogEntry(
    epoch=28,
    training_loss=3.608847066953585,
    loss_statistics={'crossent_loss_sum': 0.015952076368960025, 'center_loss_sum': 0.00015884808405926227},
    training_accuracy=0.034090906381607056,
    validation_equal_error_rate=0.034090906381607056,
}


 --- Starting Epoch 29 of 30 ---

Training:
TrainingLogEntry(
    epoch=28,
    training_loss=3.608847066953585,
    loss_statistics={'crossent_loss_sum': 0.015952076368960025, 'center_loss_sum': 0.00015884808405926227},
    training_accuracy=0.034090906381607056,
    validation_equal_error_rate=0.034090906381607056,
}


 --- Starting Epoch 29 of 30 ---

Training:


100%|██████████| 77/77 [01:13<00:00,  1.05it/s]



Average Loss: 3.5670240947178433
Multiclass accuracy: 0.04220779240131378
TrainingLogEntry(
    epoch=29,
    training_loss=3.5670240947178433,
    loss_statistics={'crossent_loss_sum': 0.015237351116235087, 'center_loss_sum': 0.00013775268900744113},
    training_accuracy=0.04220779240131378,
    validation_equal_error_rate=0.04220779240131378,
}


 --- Starting Epoch 30 of 30 ---

Training:
TrainingLogEntry(
    epoch=29,
    training_loss=3.5670240947178433,
    loss_statistics={'crossent_loss_sum': 0.015237351116235087, 'center_loss_sum': 0.00013775268900744113},
    training_accuracy=0.04220779240131378,
    validation_equal_error_rate=0.04220779240131378,
}


 --- Starting Epoch 30 of 30 ---

Training:


100%|██████████| 77/77 [01:14<00:00,  1.03it/s]



Average Loss: 3.558208886679117
Multiclass accuracy: 0.04707792028784752
TrainingLogEntry(
    epoch=30,
    training_loss=3.558208886679117,
    loss_statistics={'crossent_loss_sum': 0.014708289000895117, 'center_loss_sum': 0.00011758134323394014},
    training_accuracy=0.04707792028784752,
    validation_equal_error_rate=0.04707792028784752,
}

Stage 2 Complete! Final model saved to: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage2_crossmatch

Training Summary:
  Stage 1: Trained on 6000 SOCOFing samples
  Stage 2: Fine-tuned on 616 Crossmatch samples
  Final model location: c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\stage2_crossmatch/best_model.pyt

Next steps:
  1. Evaluate the model on held-out test data
  2. Extract embeddings for your fingerprint database
  3. Use for fingerprint matching/verification
TrainingLogEntry(
    epoch=30,
    training_loss=3.558208886679117,


## Optional: Quick Test Before Full Training

Before running the full 70-epoch training (40 + 30), you can test with fewer epochs to verify everything works:
- Stage 1: 5 epochs on SOCOFing
- Stage 2: 5 epochs on Crossmatch

This will take ~1-2 hours and help you catch any issues early.

In [6]:
# QUICK TEST: Run with reduced epochs (comment out if doing full training)
# Uncomment the lines below to run a quick test:

TEST_STAGE1_DIR = os.path.abspath("trained_models/test_stage1")
TEST_STAGE2_DIR = os.path.abspath("trained_models/test_stage2")
os.makedirs(TEST_STAGE1_DIR, exist_ok=True)
os.makedirs(TEST_STAGE2_DIR, exist_ok=True)

# Quick test Stage 1
test_extractor1 = get_DeepPrint_Tex(
    num_training_subjects=soco_loader.ids.num_subjects,
    num_texture_dims=512
)
test_extractor1.fit(
    fingerprints=soco_fingerprints,
    minutia_maps=None,
    labels=soco_labels,
    validation_fingerprints=None,
    validation_benchmark=None,
    num_epochs=5,  # Just 5 epochs for testing
    out_dir=TEST_STAGE1_DIR
)

# Quick test Stage 2
test_extractor2 = get_DeepPrint_Tex(
    num_training_subjects=crossmatch_loader.ids.num_subjects,
    num_texture_dims=512
)
test_extractor2.load_best_model(TEST_STAGE1_DIR)
test_extractor2.fit(
    fingerprints=crossmatch_fingerprints,
    minutia_maps=None,
    labels=crossmatch_labels,
    validation_fingerprints=None,
    validation_benchmark=None,
    num_epochs=5,  # Just 5 epochs for testing
    out_dir=TEST_STAGE2_DIR
)

print("\nQuick test complete! If successful, run the full training above.")

print("Quick test code is commented out. Uncomment to run a 5+5 epoch test first.")

Using device cuda:0
Loaded existing model from c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\test_stage1\model.pyt
Loaded existing model from c:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\notebooks\trained_models\test_stage1\model.pyt


C:\Users\koechian\Documents\Projects\fixed-length-fingerprint-extractors\flx\models\torch_helpers.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.lo



 --- Starting Epoch 4 of 5 ---

Training:


  0%|          | 0/750 [00:33<?, ?it/s]



KeyboardInterrupt: 

## Important Notes

### Training Time Expectations
- **Stage 1 (40 epochs)**: ~7-10 hours on GPU, ~40-60 hours on CPU
- **Stage 2 (30 epochs)**: ~2.5-5 hours on GPU, ~15-25 hours on CPU
- **Total**: ~10-15 hours on GPU

### What to Monitor During Training

**Stage 1 Metrics:**
- `training_loss`: Should steadily decrease
- `training_accuracy`: Should increase (aim for >90%)
- `crossent_loss`: Classification loss (should decrease)
- `center_loss`: May not decrease much (only 1 impression per subject)

**Stage 2 Metrics:**
- `training_loss`: Should continue to decrease
- `training_accuracy`: Should remain high or increase
- `center_loss`: Should decrease significantly (multiple impressions per subject)
- This is where the model learns intra-class similarity!

### Troubleshooting

**If training loss increases:**
- Learning rate might be too high
- Reduce from 0.025 to 0.01 in `flx/setup/config.py`

**If accuracy plateaus early:**
- Normal for Stage 1 around 85-95%
- Should improve in Stage 2 with fine-tuning

**If you run out of memory:**
- Reduce batch size in the dataloader settings
- Use a smaller embedding size (e.g., 256 instead of 512)

### After Training

The trained model will be saved in:
- Stage 1: `trained_models/stage1_soco/best_model.pyt`
- Stage 2: `trained_models/stage2_crossmatch/best_model.pyt`

Use the Stage 2 model for production - it has both discriminative features AND intra-class similarity.